In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

In [2]:
!pip install --upgrade threadpoolctl


In [ ]:
!pip uninstall scikit-learn


In [ ]:
!pip install scikit-learn

In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.pyplot as plt

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

# Load data from a CSV file
data = pd.read_csv('Resources/crypto_market_data.csv')

# Select only numeric columns for scaling
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit and transform the numeric data
normalized_data = scaler.fit_transform(numeric_data)

# Convert the normalized data back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=numeric_data.columns)

# Optionally, merge normalized data with non-numeric columns
non_numeric_data = data.select_dtypes(exclude=['float64', 'int64'])
final_df = pd.concat([normalized_df, non_numeric_data], axis=1)

# Optionally, save the normalized data to a new CSV file
final_df.to_csv('normalized_crypto_market_data.csv', index=False)

In [ ]:
# Create a DataFrame with the scaled data
normalized_df = pd.DataFrame(normalized_data, columns=numeric_data.columns)

# Copy the crypto names from the original DataFrame
crypto_names = data['coin_id']

# Set the coin_id column as index
normalized_df['coin_id'] = crypto_names
normalized_df.set_index('coin_id', inplace=True)

# Display the scaled DataFrame
print(normalized_df)

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
print(k_values)


In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Reduce the number of clusters to 3 (or any number <= 5)
k = 3

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop: 
for k in range(1, 11):
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=k, random_state=0)
    
    # 2. Fit the model to the data using `normalized_df`
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(normalized_df)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia = []
    inertia.append(kmeans.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': range(1, len(inertia) + 1), 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

# Print the DataFrame to verify
print(elbow_df)


In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.

plt.plot(elbow_df['k'], elbow_df['inertia'], marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 
The best value for k is typically identified using the Elbow method. This involves plotting the inertia (sum of squared distances from each point to its assigned cluster center) for different values of k, and then visually inspecting the plot to find the "elbow point." The elbow point is where the rate of decrease sharply slows down, indicating diminishing returns for increasing k.

In your case, you can plot the inertia values using the code provided and look for the point where the plot forms an elbow. The optimal k is generally the value at this elbow point.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [ ]:
# Set the optimal number of clusters (k) determined from the Elbow curve
best_k = 4  

# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=best_k, random_state=42)

In [ ]:
# Fit the K-Means model using the scaled DataFrame
kmeans.fit(X_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
cluster_labels = kmeans.predict(X_scaled)

# Print the resulting array of cluster values.
print("Cluster labels:", cluster_labels)

In [ ]:
# Create a copy of the scaled DataFrame
scaled_df_copy = scaled_df.copy()

print(scaled_df_copy.head())


In [ ]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
scaled_df_copy['Cluster'] = cluster_labels

# Display the copy of the scaled DataFrame
print(scaled_df_copy.head())

In [ ]:
# Create a scatter plot using hvPlot by setting
scatter_plot = scaled_df_copy.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d",
    c="Cluster",  # Color the graph points with the labels found using K-Means and
    hover_cols=["crypto_name"]  # add the crypto name in the `hover_cols` parameter to identify
) 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.

    


# the cryptocurrency represented by each data point.


---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
pca_data = pca.fit_transform(X_scaled)

# down to three principal components.
pca_df = pd.DataFrame(
    data=pca_data, 
    columns=['PC1', 'PC2', 'PC3']  # Naming the principal components
)

# View the scaled PCA data
print(pca_df.head())

In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca.explained_variance

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [ ]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(pca_data)

# Copy the crypto names from the original scaled DataFrame
pca_df["coin_id"] = scaled_df["coin_id"]

# Set the coin_id column as index
pca_df.set_index("coin_id", inplace=True)

# Display the scaled PCA DataFrame
print(pca_df)

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for k in range(1, 12):
    
# 1. Create a KMeans model using the loop counter for the n_clusters
     kmeans = KMeans(n_clusters=k, random_state=42)
    
# 2. Fit the model to the data using `df_market_data_pca`
kmeans.fit(df_market_data_pca)

# 3. Append the model.inertia_ to the inertia list
   inertia.append(kmeans.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

print(elbow_df)

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.

plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o', linestyle='-', color='b')

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** 

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [ ]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters=optimal_k, random_state=42)

In [ ]:
# Fit the K-Means model using the PCA data
kmeans_model.fit(df_market_data_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
predicted_clusters = kmeans_model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print(predicted_clusters)

In [ ]:
# Create a copy of the scaled PCA DataFrame
pca_scaled_copy = df_market_data_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
pca_scaled_copy["Cluster"] = predicted_clusters

# Display the copy of the scaled PCA DataFrame
print(pca_scaled_copy)

In [ ]:
# Create a scatter plot using hvPlot by setting
scatter_plot = pca_scaled_copy.hvplot.scatter(
    x="PC1", 
    y="PC2", 
    c="Cluster",  # Color the graph points with the labels found using K-Means and
    hover_cols=["coin_id"],  # add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.
    title="Cryptocurrency Clusters"
)

# Display the plot
scatter_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
or dataset_name, data in datasets.items():
    plt.plot(data["k_values"], data["inertia"], marker='o', label=dataset_name)

In [ ]:
# Composite plot to contrast the clusters
for ax, (dataset_name, data) in zip(axes, datasets.items()):
    ax.scatter(
        data["PC1"], 
        data["PC2"], 
        c=data["Cluster"], 
        cmap="viridis", 
        s=100, 
        alpha=0.7
    )
    ax.set_title(dataset_name, fontsize=12)
    ax.set_xlabel("PC1", fontsize=10)
    ax.set_ylabel("PC2", fontsize=10)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact is both positive and negative. The positive impact includes (a)Improved Performance, (b) reduced risk of overfitting, and (c) simplified visualiztion. On the other hand, the negative impact includes (a) loss of Information, (b) over simplification of the data, and (c) bias in results.

The impact also includes a trade off which involves balancing interpretability and computational efficiency against the need to retain meaningful patterns in the data. 